# asyncio
#### the exciting world of concurrent Python coroutines…

This notebook is a companion to a [talk on asyncio](https://speakerdeck.com/jacobtomlinson/asyncio-deep-dive).

### Install deps

In [ ]:
!conda install aiohttp -y

## 1.2 Generators - value = yield

In [ ]:
def jumping_range(up_to):
    index = 0
    while index < up_to:
        jump = yield index
        if jump is None:
            jump = 1
        index += jump

if __name__ == '__main__':
    iterator = jumping_range(5)
    print(next(iterator))        # 0
    print(iterator.send(2))      # 2
    print(next(iterator))        # 3
    print(iterator.send(-1))     # 2
    for x in iterator:
        print(x)                 # 3, 4

## 2.1 coroutines

In [ ]:
import time

def countdown(number, n):
    while n > 0:
        print('T-minus', n, '({})'.format(number))
        time.sleep(1)
        n -= 1

countdown("A", 2)
countdown("B", 3)

In [ ]:
import asyncio

@asyncio.coroutine
def countdown(number, n):
    while n > 0:
        print('T-minus', n, '({})'.format(number))
        yield from asyncio.sleep(1)
        n -= 1

loop = asyncio.get_event_loop()
tasks = [
    asyncio.ensure_future(countdown("A", 2)),
    asyncio.ensure_future(countdown("B", 3))]
loop.run_until_complete(asyncio.wait(tasks))

## 4.1.1 aiohttp client

In [ ]:
import aiohttp
import asyncio
import async_timeout

async def fetch(session, url):
    with async_timeout.timeout(10):
        async with session.get(url) as response:
            return await response.text()
        
async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://ipecho.net/plain')
        print(html)
        
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

## 5. run_in_executor - blocking

In [ ]:
import asyncio
import time
    
async def my_blocking_coroutine():
    print("Blocking started")
    time.sleep(4)
    print("Blocking done")
    
async def my_nonblocking_coroutine():
    print("Nonblocking started")
    await asyncio.sleep(2)
    print("Nonblocking done")
    
loop = asyncio.get_event_loop()
tasks = [
    asyncio.ensure_future(my_nonblocking_coroutine()),
    asyncio.ensure_future(my_blocking_coroutine())]
loop.run_until_complete(asyncio.wait(tasks))

## 5. run_in_executor - nonblocking

In [ ]:
import asyncio
import time
    
async def my_blocking_coroutine():
    print("Blocking started")
    await loop.run_in_executor(None, time.sleep, 4)
    print("Blocking done")
    
async def my_nonblocking_coroutine():
    print("Nonblocking started")
    await asyncio.sleep(2)
    print("Nonblocking done")
    
loop = asyncio.get_event_loop()
tasks = [
    asyncio.ensure_future(my_nonblocking_coroutine()),
    asyncio.ensure_future(my_blocking_coroutine())]
loop.run_until_complete(asyncio.wait(tasks))